<a href="https://colab.research.google.com/github/nbacomdados/nbacomdados/blob/main/notebooks/leitura_jogos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
import sys

In [3]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [4]:
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [5]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [6]:
url_nba = 'https://www.nba.com/game/mil-vs-bkn-0042000217/box-score'

In [7]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.get(url_nba)
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "__next"))
    )
finally:
    the_text = driver.page_source
    driver.quit()

In [8]:
import pandas as pd
from io import StringIO

In [9]:
the_text_io = StringIO(the_text)

In [10]:
dfs_nba = pd.read_html(the_text_io)
len(dfs_nba)

2

In [11]:
dfs_nba

[                                    PLAYER  ...                     +/-
 0                    P.J. TuckerP. TuckerF  ...                      -4
 1   Giannis AntetokounmpoG. AntetokounmpoF  ...                       6
 2                     Brook LopezB. LopezC  ...                       2
 3             Khris MiddletonK. MiddletonG  ...                       3
 4                  Jrue HolidayJ. HolidayG  ...                       3
 5   Thanasis AntetokounmpoT. Antetokounmpo  ...                      -2
 6            Pat ConnaughtonP. Connaughton  ...                      10
 7                     Bryn ForbesB. Forbes  ...                       2
 8                   Elijah BryantE. Bryant  ...  DNP - Coach's Decision
 9                 Mamadi DiakiteM. Diakite  ...  DNP - Coach's Decision
 10                Justin JacksonJ. Jackson  ...  DNP - Coach's Decision
 11                   Sam MerrillS. Merrill  ...  DNP - Coach's Decision
 12                   Bobby PortisB. Portis  ...  D

In [12]:
import requests

In [16]:
url_stats = 'https://stats.nba.com/stats/boxscoretraditionalv3?GameID=0042000217&LeagueID=00&endPeriod=0&endRange=7727&rangeType=2&startPeriod=0&startRange=0'

In [17]:
headers: dict = {'Accept': '*/*',
                'Accept-Encoding': 'gzip, deflate, br',
                'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
                'Connection': 'keep-alive',
                'Host': 'stats.nba.com',
                'Origin': 'https://www.nba.com',
                'Referer': 'https://www.nba.com/',
                'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
                'sec-ch-ua-mobile': '?0',
                'Sec-Fetch-Dest': 'empty',
                'Sec-Fetch-Mode': 'cors',
                'Sec-Fetch-Site': 'same-site',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

In [18]:
with requests.Session() as sessao:
        resposta: requests.models.Response = sessao.get(url_stats, headers=headers)

In [22]:
x = resposta.content

In [23]:
import json

In [24]:
y = json.loads(x)

In [32]:
y.keys()

dict_keys(['meta', 'boxScoreTraditional'])

In [35]:
pd.json_normalize(y['boxScoreTraditional']['awayTeam']['players'])

,personId,firstName,familyName,nameI,playerSlug,position,comment,jerseyNum,statistics.minutes,statistics.fieldGoalsMade,statistics.fieldGoalsAttempted,statistics.fieldGoalsPercentage,statistics.threePointersMade,statistics.threePointersAttempted,statistics.threePointersPercentage,statistics.freeThrowsMade,statistics.freeThrowsAttempted,statistics.freeThrowsPercentage,statistics.reboundsOffensive,statistics.reboundsDefensive,statistics.reboundsTotal,statistics.assists,statistics.steals,statistics.blocks,statistics.turnovers,statistics.foulsPersonal,statistics.points,statistics.plusMinusPoints
0,200782,P.J.,Tucker,P. Tucker,pj-tucker,F,,,9:23,1,2,0.50,1,1,1.0,0,0,0.0,0,1,1,0,0,2,0,1,3,-3.0
1,203507,Giannis,Antetokounmpo,G. Antetokounmpo,giannis-antetokounmpo,F,,,12:04,4,5,0.80,1,2,0.5,1,2,0.5,1,1,2,1,0,1,1,1,10,-6.0
2,201572,Brook,Lopez,B. Lopez,brook-lopez,C,,,12:09,1,2,0.50,1,2,0.5,0,0,0.0,1,1,2,1,0,0,0,0,3,-4.0
3,203114,Khris,Middleton,K. Middleton,khris-middleton,G,,,12:07,2,8,0.25,0,1,0.0,1,1,1.0,3,2,5,1,0,0,1,0,5,-7.0
4,201950,Jrue,Holiday,J. Holiday,jrue-holiday,G,,,12:00,2,8,0.25,0,3,0.0,0,0,0.0,1,0,1,1,0,0,1,1,4,-3.0
5,203648,Thanasis,Antetokounmpo,T. Antetokounmpo,thanasis-antetokounmpo,,,,0:42,0,0,0.00,0,0,0.0,0,0,0.0,0,1,1,0,0,0,0,1,0,0.0
6,1626192,Pat,Connaughton,P. Connaughton,pat-connaughton,,,,2:47,0,1,0.00,0,1,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,-3.0
7,1627854,Bryn,Forbes,B. Forbes,bryn-forbes,,,,2:38,0,0,0.00,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,-4.0
